# **Inspiration**
* Trying to validate the established idea of human sensitiveness to price drops ? (discounted price compared to original retail_price)
* Do bad products sell ? How about the relationship between the quality of a product (ratings) and its success ? Does the price factor into this?
* Do seller's fame factor into top products ?
* Do the number of tags (making a product more discoverable) factor into the success of a product ?

# Reading The Dataseta

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df1 = pd.read_csv('/kaggle/input/summer-products-and-sales-in-ecommerce-wish/unique-categories.csv')
df2 = pd.read_csv('/kaggle/input/summer-products-and-sales-in-ecommerce-wish/unique-categories.sorted-by-count.csv')
df = pd.read_csv('/kaggle/input/summer-products-and-sales-in-ecommerce-wish/summer-products-with-rating-and-performance_2020-08.csv')

In [ ]:
pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)

In [ ]:
df.head(3)

In [ ]:
#columns =['title','title_orig'] #,'merchant_profile_picture','product_url','product_picture','merchant_name','merchant_title']
#df = dff.drop(columns, axis=1)
df.columns

The column names raises some more possible analysis that can be performed:
* How different is the 'price' from 'retail price' and what is the effect of the difference. Expecting higher number of units sold if price is less than retail price and vice versa.
* Does having ad boosts increase success?
* Any correlation between units sold and ratings?
* What is badges? This looks like 'awards' of some sort? Do success with increase in number of badges?
* What are effect of different type of badges?
* Effect of tags? Higher number of tags results in success?
* Maybe some brief analysis on product variations.. Do increased variations leads to inreased success?
* Shipping options analysis
* Analysis of inventory total and units sold.
* Merchant detail analysis.. Does merchant rating affect success?
* What about the details of the merchant? Does not having a profile picture reduce success? Perhaps detailed info leads to higher success?
* Do all product contains pictures?
* I am expecting only one theme..'summer' but let's see.


In [ ]:
df.shape

In [ ]:
round(df.describe().T)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# **Let's see if we have any missing values**

In [ ]:
df.info()

In [ ]:
plt.figure(figsize = (15,3))
sns.heatmap(df.isnull(), cmap = 'cubehelix', cbar=False)

In [ ]:
df.isnull().sum()

* The rating count has zero null values, however individual rating count has 45 null values. Does this mean that rating count of 45 products are zero?
* Product color and size variation has 41 and 14 null values respectively. Perhaps this products have no variations?
* has_urgency banner and urgency_text have similar null values. This makes sense, however, I will convert this to zeros and ones.
* Origin countries have 17 null values. I might replace this with the mode if necessary.
* Surprisingly there are 4 missing merchant names but only one missing info subtitle.
* There are 1347 missing merchant profile pictures. I am expecting this to match with 'zeros' in 'merchant_has_profile_picture'

# Replacing NA ratings with 0

In [ ]:
df['rating_five_count'] = df['rating_five_count'].fillna(0)
df['rating_four_count'] = df['rating_four_count'].fillna(0)
df['rating_three_count'] = df['rating_three_count'].fillna(0)
df['rating_two_count'] = df['rating_two_count'].fillna(0)
df['rating_one_count'] = df['rating_one_count'].fillna(0)

In [ ]:
df['has_urgency_banner'] = df['has_urgency_banner'].replace(np.nan, 0)

In [ ]:
df.isnull().any()

In [ ]:
#df['origin_country'].value_counts().plot(kind = 'bar')
sns.countplot('origin_country', data=df)

**China and US are the prominent countries. Since other countries have less contribution for the set, those items were replaced with 'Other'**

In [ ]:
df['origin_country'] = df['origin_country'].replace('VE', 'Other')
df['origin_country'] = df['origin_country'].replace('AT', 'Other')
df['origin_country'] = df['origin_country'].replace('SG', 'Other')
df['origin_country'] = df['origin_country'].replace('GB', 'Other')
df['origin_country'] = df['origin_country'].replace(np.nan, 'Other')

In [ ]:
#df['origin_country'].value_counts().plot(kind = 'bar')
sns.countplot('origin_country', data=df)

# Product Variation - Data Analysis

In [ ]:
# Replacing different variations of sizes into one.

df['product_variation_size_id'] = df['product_variation_size_id'].replace('S.', 'S')
df['product_variation_size_id'] = df['product_variation_size_id'].replace('XS.', 'S')
df['product_variation_size_id'] = df['product_variation_size_id'].replace('M.', 'M')

df['product_variation_size_id'] = df['product_variation_size_id'].replace('Size S', 'S')
df['product_variation_size_id'] = df['product_variation_size_id'].replace('Size-XS', 'S')
df['product_variation_size_id'] = df['product_variation_size_id'].replace('SIZE XS', 'S')
df['product_variation_size_id'] = df['product_variation_size_id'].replace('Size-S', 'S')
df['product_variation_size_id'] = df['product_variation_size_id'].replace('Size4XL', 'XL')
df['product_variation_size_id'] = df['product_variation_size_id'].replace('size S', 'S')
df['product_variation_size_id'] = df['product_variation_size_id'].replace('Size M', 'M')
df['product_variation_size_id'] = df['product_variation_size_id'].replace('Size -XXS', 'S')
df['product_variation_size_id'] = df['product_variation_size_id'].replace('SIZE-XXS', 'S')
df['product_variation_size_id'] = df['product_variation_size_id'].replace('Size S.', 'S')
df['product_variation_size_id'] = df['product_variation_size_id'].replace('s', 'S')
df['product_variation_size_id'] = df['product_variation_size_id'].replace('SizeL', 'L')

df['product_variation_size_id'] = df['product_variation_size_id'].replace('5XL', 'XL')
df['product_variation_size_id'] = df['product_variation_size_id'].replace('4XL', 'XL')
df['product_variation_size_id'] = df['product_variation_size_id'].replace('3XL', 'XL')
df['product_variation_size_id'] = df['product_variation_size_id'].replace('2XL', 'XL')

In [ ]:
pr_val = df['product_variation_size_id'].value_counts()
#pr_val = pr_val[pr_val>10]
pr_val[pr_val > 1]

In [ ]:
def pr_var(name):
    if name == 'XXXS' \
    or name == 'XXS' \
    or name == 'XS' \
    or name == 'S' \
    or name == 'M' \
    or name == 'L' \
    or name == 'XL' \
    or name == 'XXL' \
    or name == 'XXXXL' \
    or name == 'XXXXXL':
        return name
    else:
        return "OTHER"
    
df['product_variation_size_id'] = df['product_variation_size_id'].replace(np.nan, 'OTHER')
df['product_variation_size_id'] = df['product_variation_size_id'].apply(pr_var)

In [ ]:
fig_dims = (10, 10)
fig, ax = plt.subplots(figsize=fig_dims)
sns.countplot('product_variation_size_id',
              order = df['product_variation_size_id'].value_counts().index,
              palette="magma",
              data = df,
              ax = ax)
ax.set(xlabel='Size', ylabel='Count')
plt.show()

# Product Color - Data Analysis

In [ ]:
pr_colr = df['product_color'].value_counts()
pr_colr[pr_colr > 2]

In [ ]:
df['product_color'] = df['product_color'].replace('Black', 'black')
df['product_color'] = df['product_color'].replace('White', 'white')

df['product_color'] = df['product_color'].replace('navyblue', 'blue')
df['product_color'] = df['product_color'].replace('lightblue', 'blue')
df['product_color'] = df['product_color'].replace('skyblue', 'blue')
df['product_color'] = df['product_color'].replace('darkblue', 'blue')
df['product_color'] = df['product_color'].replace('navy', 'blue')

df['product_color'] = df['product_color'].replace('winered', 'red')
df['product_color'] = df['product_color'].replace('rosered', 'red')
df['product_color'] = df['product_color'].replace('rose', 'red')
df['product_color'] = df['product_color'].replace('orange-red', 'red')

df['product_color'] = df['product_color'].replace('lightpink', 'pink')

df['product_color'] = df['product_color'].replace('armygreen', 'green')
df['product_color'] = df['product_color'].replace('khaki', 'green')
df['product_color'] = df['product_color'].replace('lightgreen', 'green')
df['product_color'] = df['product_color'].replace('fluorescentgreen', 'green')

df['product_color'] = df['product_color'].replace('gray', 'grey')

df['product_color'] = df['product_color'].replace('coffee', 'brown')

df['product_color'] = df['product_color'].replace('multicolor', 'other')
df['product_color'] = df['product_color'].replace('floral', 'other')
df['product_color'] = df['product_color'].replace('leopard', 'other')
df['product_color'] = df['product_color'].replace('camouflage', 'other')

df['product_color'] = df['product_color'].replace('white & green', 'dual')
df['product_color'] = df['product_color'].replace('black & green', 'dual')
df['product_color'] = df['product_color'].replace('black & white', 'dual')
df['product_color'] = df['product_color'].replace('camouflage', 'other')
    
df['product_color'] = df['product_color'].replace(np.nan, 'other')

pr_colr = df['product_color'].value_counts()
pr_colr[pr_colr > 2]

In [ ]:
# Show Most popular colors
fig_dims = (10, 15)
fig, ax = plt.subplots(figsize=fig_dims)
sns.countplot('product_color',
              data = df,
              order = df['product_color'].value_counts().iloc[:15].index,
              ax = ax)
ax.set(xlabel='Product Colour', ylabel='Count')
plt.xticks(rotation=45, ha='right')
plt.show()

# Units Sold - Data Analysis

In [ ]:
print('Median of units sold is',df['units_sold'].median())
print('Mean of units sold is',df['units_sold'].mean())
df['units_sold'].value_counts()

In [ ]:
def below_ten(units_sold):
    if units_sold < 10:
        return 10
    else:
        return units_sold

In [ ]:
df['units_sold'] = df['units_sold'].apply(below_ten)

In [ ]:
df['units_sold'].value_counts()

In [ ]:
df[df['units_sold'] == 100000]

In [ ]:
def is_successful(units_sold):
    if units_sold > 1000:
        return 1
    else:
        return 0

In [ ]:
df['is_successful'] = df['units_sold'].apply(is_successful)
#df['is_successful'] = df['units_sold'].apply(is_successful).astype('category')
print('Percent of successful products: ', df['is_successful'].value_counts()[1] / len(df['is_successful'])*100)
sns.countplot(data=df, x='is_successful')
plt.show()

# Popular product tags in Summer - 2020

In [ ]:
fig_dims = (15, 10)
fig, ax = plt.subplots(figsize=fig_dims)
sns.barplot(x = 'keyword',
            y = 'count',
            data = df2.iloc[:20],
            ax = ax)
ax.set(xlabel='Keyword', ylabel='Count')
plt.xticks(rotation=45, ha='right')
plt.show()

**Now let's create a column which indicates how many popular tags contain on a specific product tag**

In [ ]:
pop_tags = df2.iloc[:10]['keyword'].tolist()

def check_tag_quality(tags):
    tag_str = tags
    prod_tags = tag_str.split(',')
    
    common_elements = np.intersect1d(pop_tags, prod_tags)
    return len(common_elements) / len(pop_tags)
    
df['tag_quality'] = df['tags'].apply(check_tag_quality)

**Check the number of tags include in the product**

In [ ]:
def tag_count(tags):
    tag_str = tags
    prod_tags = tag_str.split(',')
    return len(prod_tags)
    
df['tag_count'] = df['tags'].apply(tag_count)

**Check whether the title was localised or not**

In [ ]:
def is_localised(trans,origi):
    if trans == origi:
        return 0
    else:
        return 1
    
df['title_localised'] = df.apply(lambda x: is_localised(x.title, x.title_orig), axis = 1)

**Check Merchant title and Merchant name**

In [ ]:
def check_name(title,name):
    title = str(title).replace(" ", "")
    title = title.replace(".", "")
    
    name = str(name).replace(" ", "")
    name = name.replace(".", "")
    
    if title.lower() == name.lower():
        return 1
    else:
        return 0
    
df['com_same_name'] = df.apply(lambda x: check_name(x.merchant_title, x.merchant_name), axis = 1)

**Check whether the merchant subtitle contains postive ratings**

In [ ]:
def inc_positive_per(wordset):
    pos_ses = 'avis positifs'
    if f'{pos_ses}' in f'{wordset}':
        return 1
    else:
        return 0

df['subtitle_rating'] = df['merchant_info_subtitle'].apply(inc_positive_per)

In [ ]:
def check_rating_prop(s_count,t_count):
    if s_count == 0:
        return 0
    else:
        return s_count/t_count
    
df['rating_five_count'] = df.apply(lambda x: check_rating_prop(x.rating_five_count, x.rating_count), axis = 1)
df['rating_four_count'] = df.apply(lambda x: check_rating_prop(x.rating_four_count, x.rating_count), axis = 1)
df['rating_three_count'] = df.apply(lambda x: check_rating_prop(x.rating_three_count, x.rating_count), axis = 1)
df['rating_two_count'] = df.apply(lambda x: check_rating_prop(x.rating_two_count, x.rating_count), axis = 1)
df['rating_one_count'] = df.apply(lambda x: check_rating_prop(x.rating_one_count, x.rating_count), axis = 1)

In [ ]:
def check_dis_per(r_price,buyer_price):
    return (r_price - buyer_price) / r_price
    
df['discount_rate'] = df.apply(lambda x: check_dis_per(x.retail_price, x.price),axis = 1)

In [ ]:
def merchant_pop_on_prod(m_count,p_count):
    if m_count == 0:
        return 0
    else:
        return p_count / m_count
    
df['merch_pop_proc'] = df.apply(lambda x: merchant_pop_on_prod(x.merchant_rating_count, x.rating_count),axis = 1)

**Correlation Heat Map**

In [ ]:
fig_dims = (25, 25)
fig, ax = plt.subplots(figsize=fig_dims)
sns.heatmap(df.corr(),
            annot=True,
            ax=ax)
ax.add_patch(plt.Rectangle((0,2),31,1, 
                           fill = False, 
                           edgecolor = 'blue', 
                           lw = 3))
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=45, ha='right')
plt.show()

Major Factors which increases the 'units_sold'

Rating Count,
Rating,
Rating Five Count,
Rating Four Count,
Rating Three Count.
Merchant Rating Count,
Merchant Rating,
Merchant Has Profile Picture,
Is Title Localised,
Rating,
Badge Product Quality,
Badges Count,
Product Variation Inventory,
Number of Tags,
Title Localised,
Number Of Tags,
Have Rating on Merchant Subtitle,
High Retail Price,
High Buyer Price,

Major Factors which decreases the 'units_sold'

Rating Two Count,
Rating One Count,
Price Increment,
Shipping Price Increment,
If Merchant title and Merchant name has differnt names,
No Urgency Banner,
High Price (Buyer Price),
Lower discount rate (Buyer price compare to retail price),

Special Note :-

Add Boost does not have any significant change to the 'units_sold'

# Price and retail price and the relation of their difference..

In [ ]:
print('Overall stats:')
print(df['price'].mean())
print(df['retail_price'].mean())
print('----------------------')
print('Stats for successful products:')
print(df[df['is_successful'] == 1]['price'].mean())
print(df[df['is_successful'] == 1]['retail_price'].mean())
print('----------------------')
print('Stats for unsuccessful products:')
print(df[df['is_successful'] == 0]['price'].mean())
print(df[df['is_successful'] == 0]['retail_price'].mean())

Maybe comparing the % change of price and retail price to the success of the products shade some light?

In [ ]:
df['diff_in_price'] = round(df['price']/df['retail_price'],2)
df['diff_in_price']

In [ ]:
sns.violinplot(data=df, y='diff_in_price', x='is_successful')

Nothing interesting here. I will just drop these columns if need be..

# The use of ad boosts to boost the success..

There is an almost equal use of ad boosts by the products.

In [ ]:
print('Percent of products using ad boosts: ', df['uses_ad_boosts'].value_counts()[1] / len(df['uses_ad_boosts'])*100)

In [ ]:
sns.countplot(data=df, x='uses_ad_boosts', hue='is_successful')

In [ ]:
pd.crosstab(df['uses_ad_boosts'], df['is_successful'])

In [ ]:
pd.crosstab(df['uses_ad_boosts'], df['units_sold'])

It seems like ad boosts do not have any impact on units sold.. perhaps something I am missing here? I mean, there must be some benefit of the addition cost(?) for the ad boosts.

The website needs to consider the effectiveness of this, otherwise the merchants may simply not use it and the website might lose this additional income.

# Higher ratings means higher units sold?

In [ ]:
df.groupby('is_successful').mean()[ratings_column]

In [ ]:
df.groupby('units_sold').mean()[ratings_column]

Successful products have more ratings. This is expected as units sold is higher so rating is also higher. However, if I am building a model, I don't think it would be a good idea to keep this column as the huge number of ratings must be the result of higher units sold.

# Successful merchants

In [ ]:
merchant_columns = ['merchant_rating_count', 'merchant_rating',
       'merchant_id', 'merchant_has_profile_picture']


sns.countplot(data=df, x='merchant_has_profile_picture', hue='is_successful')

There is a higher chance of success if merchant has profile picture.

In [ ]:
sns.violinplot(data=df, y='merchant_rating', x='is_successful')

In [ ]:
sns.violinplot(data=df, y='merchant_rating', x='units_sold')

From the above plots, the higher units sold tend to have higher ratings. which again makes sense

# Remove Unwanted Columns

In [ ]:
column_to_drop  = ['crawl_month','product_id','product_picture', 'product_url','merchant_profile_picture', 
        'merchant_id', 'currency_buyer','title','title_orig','tags','shipping_option_name','rating_count',
       'theme','urgency_text', 'merchant_title', 'merchant_name', 'merchant_info_subtitle',
       ]

In [ ]:
df = df.drop(column_to_drop, axis = 1)

# Create Model to predict Units Sold

In [ ]:
df = pd.get_dummies(df, 
                    columns = ['product_color'],
                    prefix = 'color_',
                    drop_first = True)
df.head()

In [ ]:
df = pd.get_dummies(df, columns = ['product_variation_size_id'],
                    prefix = 'size_',drop_first = True)
df.head()

In [ ]:
y = df['is_successful']
X = df[['price', 'retail_price', 'uses_ad_boosts', 'badge_product_quality',
       'product_variation_inventory',
       'inventory_total', 'merchant_rating_count', 'merchant_rating',
       'merchant_has_profile_picture','diff_in_price']]

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from xgboost.sklearn import XGBClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=57, shuffle=True)

In [ ]:
models = [LogisticRegression(), DecisionTreeClassifier(), XGBClassifier(),  
          GradientBoostingClassifier(), KNeighborsClassifier(), RandomForestClassifier()]

for model in models:
    
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    print(model)
    print('---------------------------')
    print(metrics.classification_report(y_test,y_pred))
    print('')
    print('')

I am finally going with Random Forests

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42, shuffle=True)


models = RandomForestClassifier()

model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print(metrics.accuracy_score(y_test,y_pred))

# Some conclusions
* The site mainly sells female clothing.
* Higher units sold means higher rating count.
* The use of ad boosts does not seen to have any effect on the units sold and the site may lose revenue from this ads.
* More detailed units sold and inventory levels would have been more helpful for analysis.
* Product quality badges seems to increase the success of the products. Perhaps reviewing more products will increase motivation for merchants to improve their product? Different levels of badges can be applied?
* The tags can be improved so that products can be categorised more specfically. This can be done by reducing the number of tags per product, so the mechants are forced to choose their tags more wisely.
* Majority of the products are black and white. This might have been defined wrongly by the merchants. If not the case, the merchants can be encoured to include more variation to these.